In [22]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
from itertools import product


In [2]:
sys.path.append("C:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble")

In [3]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/turns.csv', index_col='game_id')

In [4]:
import functions
import transformers
import importlib

importlib.reload(functions)
importlib.reload(transformers)
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer





from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)





<module 'transformers.series_from_group_transformer' from 'C:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\transformers\\series_from_group_transformer.py'>

In [5]:
names = ['BetterBot', 'STEEBot', 'HastyBot']
bots = BotExtarctor(names)(train)
is_player = ~IsBotExtarctor(names)(train)


In [6]:
games

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
game_id,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
2,Super,regular,STANDARD,1,2022-08-10 19:19:59,CSW21,3600,0,RATED,1,364.214418
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722
...,...,...,...,...,...,...,...,...,...,...,...
72769,browndav,blitz,STANDARD,0,2022-09-07 13:24:13,CSW21,180,0,RATED,1,218.043751
72770,HastyBot,regular,TIME,0,2022-09-04 00:13:24,CSW21,3600,0,RATED,1,3662.659287
72771,HastyBot,regular,STANDARD,0,2022-09-19 03:28:44,ECWL,900,0,CASUAL,10,236.133634


In [13]:
train_pipe = (Pipeline([('bot_names', ExtractSetColumnsTransformer({'bot_name': BotExtarctor(names)})),
                   ('players_selection', SelectTransformer(is_player)),
                   ('set_bot_training', ColumnsSetterTransformer({'bot_rating': train[~is_player]['rating']})),
                   ('name_dropper', NameDropperTransformer(['nickname']))
                   ]))
games_pipe = (Pipeline([('extract_first', ExtractSetColumnsTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                        ('drop_first', NameDropperTransformer('first'))
                       ]))

new_train = train_pipe.transform(train)
new_games = games_pipe.transform(games).merge(new_train, left_index=True, right_index=True)
new_games

,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,is_bot_first,score,rating,bot_name,bot_rating
game_id,,,,,,,,,,,,,,,
1,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,True,429,1500,BetterBot,1637
3,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,True,440,1811,BetterBot,2071
4,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,True,119,1473,BetterBot,1936
5,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,True,325,1500,STEEBot,1844
6,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,False,378,2029,STEEBot,2143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,regular,TIME,0,2022-09-05 04:35:56,CSW21,3600,0,CASUAL,1,3669.988413,True,340,1395,BetterBot,1952
72770,regular,TIME,0,2022-09-04 00:13:24,CSW21,3600,0,RATED,1,3662.659287,True,97,1332,HastyBot,2237
72771,regular,STANDARD,0,2022-09-19 03:28:44,ECWL,900,0,CASUAL,10,236.133634,True,390,1500,HastyBot,1614


In [15]:
new_games.corr(numeric_only=True)

,winner,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,is_bot_first,score,rating,bot_rating
winner,1.000000,0.026618,0.014037,0.086464,-0.029692,-0.094712,0.598152,0.103753,-0.109853
initial_time_seconds,0.026618,1.000000,0.183511,-0.045173,0.345675,0.001068,0.077908,0.163808,0.158251
increment_seconds,0.014037,0.183511,1.000000,-0.068178,0.100689,0.002151,0.010489,0.001256,0.011898
max_overtime_minutes,0.086464,-0.045173,-0.068178,1.000000,-0.234576,0.005080,0.031740,-0.406451,-0.538835
game_duration_seconds,-0.029692,0.345675,0.100689,-0.234576,1.000000,-0.004671,-0.003773,0.141911,0.184368
is_bot_first,-0.094712,0.001068,0.002151,0.005080,-0.004671,1.000000,-0.097253,-0.002695,-0.005289
score,0.598152,0.077908,0.010489,0.031740,-0.003773,-0.097253,1.000000,0.334759,0.075921
rating,0.103753,0.163808,0.001256,-0.406451,0.141911,-0.002695,0.334759,1.000000,0.673723
bot_rating,-0.109853,0.158251,0.011898,-0.538835,0.184368,-0.005289,0.075921,0.673723,1.000000


In [17]:
new_games.dtypes

time_control_name         object
game_end_reason           object
winner                     int64
created_at                object
lexicon                   object
initial_time_seconds       int64
increment_seconds          int64
rating_mode               object
max_overtime_minutes       int64
game_duration_seconds    float64
is_bot_first                bool
score                      int64
rating                     int64
bot_name                  object
bot_rating                 int64
dtype: object

In [31]:
cats = ['winner', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'is_bot_first', 'bot_name']
cats_games = new_games[cats]

In [30]:
cat_var_prod = list(product(cats, cats, repeat = 1))
result = []
for i in cat_var_prod:
    if i[0] != i[1]:
        result.append((i[0],i[1],list(ss.chi2_contingency(pd.crosstab(
                            cat_games[i[0]], cat_games[i[1]])))[1]))

[('winner', 'winner'),
 ('winner', 'time_control_name'),
 ('winner', 'game_end_reason'),
 ('winner', 'lexicon'),
 ('winner', 'rating_mode'),
 ('winner', 'is_bot_first'),
 ('winner', 'bot_name'),
 ('time_control_name', 'winner'),
 ('time_control_name', 'time_control_name'),
 ('time_control_name', 'game_end_reason'),
 ('time_control_name', 'lexicon'),
 ('time_control_name', 'rating_mode'),
 ('time_control_name', 'is_bot_first'),
 ('time_control_name', 'bot_name'),
 ('game_end_reason', 'winner'),
 ('game_end_reason', 'time_control_name'),
 ('game_end_reason', 'game_end_reason'),
 ('game_end_reason', 'lexicon'),
 ('game_end_reason', 'rating_mode'),
 ('game_end_reason', 'is_bot_first'),
 ('game_end_reason', 'bot_name'),
 ('lexicon', 'winner'),
 ('lexicon', 'time_control_name'),
 ('lexicon', 'game_end_reason'),
 ('lexicon', 'lexicon'),
 ('lexicon', 'rating_mode'),
 ('lexicon', 'is_bot_first'),
 ('lexicon', 'bot_name'),
 ('rating_mode', 'winner'),
 ('rating_mode', 'time_control_name'),
 ('rat